### Teste de Tesseract para PVP

In [1]:
import pytesseract
import time
from PIL import Image, ImageDraw
import util_gui
import glob
import os
import time
import cv2 as cv
import numpy as np
import pandas as pd
import pyautogui as gui
import re

In [2]:
pytesseract.pytesseract.tesseract_cmd = 'C:\Program Files\Tesseract-OCR\Tesseract'  
df_image = pd.read_csv(os.path.join(os.getcwd(),r"CSVs\data_images.csv"),index_col = 'Unnamed: 0')

In [22]:
def drawing_rec(findall,image):
    #image é do PIL Image.open
    #findall é o objeto do return do gui.locateall
    image_copy = image.copy()
    if(isinstance(findall[0],list)):
        for i in range(len(findall)):
            x1_f = findall[i][0]
            x2_f = findall[i][2]
            y1_f = findall[i][1]
            y2_f = findall[i][3]
            ImageDraw.Draw(image_copy).rectangle([x1_f,y1_f,x2_f,y2_f],fill=None,outline="red")
    else:
        print('not list')
        for i in range(len(findall)):
            x1_f = findall[i].left
            x2_f = findall[i].left+findall[i].width
            y1_f = findall[i].top
            y2_f = findall[i].top+findall[i].height
            ImageDraw.Draw(image_copy).rectangle([x1_f,y1_f,x2_f,y2_f],fill=None,outline="red")
    image_copy.show()
    return None

In [47]:
image_gen = Image.open(df_image['Path']['PVP_gen'])
image_lv = Image.open(df_image['Path']['PVP_lv'])
image_lv_triangle = Image.open(df_image['Path']['PVP_lv_triangle'])

fp_pos = []
lv_image = []
lv_triangle = []
lv_lista = []
lv_pos = []
fp_lista = []
re_pattern_lvl = "([\w]*[.])|([a-zA-Z]*)"
util_gui.wait(image_gen)
pvp = gui.screenshot()
# pvp = Image.open("C:\\Users\\EricR\\Desktop\\exemplo_pvp_2.png")
# pvp = Image.open("C:\\Users\\EricR\\Desktop\\exemplo_pvp.png")
for i ,coords in enumerate(gui.locateAll(image_gen,pvp,confidence = 0.85,region = (0,0,1080,420))):
    x,y,w,h = coords
    x,y,w,h = x-2,y+h+2,x+55,y+h+60
    fp_pos.append([x,y,w,h])
    string = pytesseract.image_to_string(pvp.crop(fp_pos[i]),config="-c tessedit_char_whitelist=0123456789")
    fp_lista.append(string.splitlines())
print(fp_lista)

lv_image = list(gui.locateAll(image_lv,pvp,grayscale=True,confidence = 0.93,region = (0,0,1080,420)))
lv_triangle = list(gui.locateAll(image_lv_triangle,pvp,confidence = 0.90,region = (0,0,1080,420)))

for i in range(len(lv_image)):
    x1 = lv_image[i].left
    y1 = lv_image[i].top+1
    x2 = lv_triangle[i].left+3
    y2 = lv_image[i].top + lv_image[i].height+2
    string = pytesseract.image_to_string(pvp.crop([x1,y1,x2,y2])).splitlines()[0]
    lv_pos.append([x1,y1,x2,y2])
    
    lv_lista.append(re.sub(re_pattern_lvl,'',string))
    lv_lista = list(map(int,lv_lista))

[['17821', '16222'], ['16360', '14161'], ['103', '14846'], ['15832', '14939']]


IndexError: list index out of range

In [43]:
drawing_rec(fp_pos,pvp)

In [60]:
cv_pvp = cv.cvtColor(np.array(pvp), cv.COLOR_RGB2GRAY)
th = cv.adaptiveThreshold(cv_pvp,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,cv.THRESH_BINARY,11,2)

In [61]:
cv.imshow('oi',th)
cv.waitKey(0)

-1

In [ ]:
pytesseract.image_to_string(pvp.crop(fp_pos[i]),config="-c tessedit_char_whitelist=0123456789")

In [ ]:
#Check for fp_lista integrity(i.e. possui 4 membros, cada um com 2 valores)
#Check for 4 membros
fp_lista_media = []
if len(fp_lista) != 4:
    print(f"{len(fp_lista)} membros apenas")
for i in fp_lista:
    #Check for both Main and Vanguard values (deve possuir 2)
    if len(i) != 2:
        print(f"Nao possui ambos valroes de Main e Vanguard")
    #Gera valores medios de fp
    fp_lista_media.append(sum([int(x) for x in i])//2)

In [ ]:
chosen_player = 0
if fp_lista_media[0] < 17000 and lv_lista[0] > 120:
    chosen_player = 0
elif fp_lista_media[1] < 17000 and lv_lista[1] > 120:
    chosen_player = 1

In [ ]:
time.sleep(1)
gui.click(fp_pos[chosen_player][0],fp_pos[chosen_player][1],button='left')

In [ ]:
confidence_imagem = 0.8
def locate_image(imagem,nome_imagem):
    while(gui.locateOnScreen(imagem,confidence = confidence_imagem) is None):
        continue
    [x,y,w,h] = gui.locateOnScreen(imagem,confidence = confidence_imagem)
    [x,y,w,h] = [0 if i < 0 else i for i in [x-10,y-10,w+10,h+10]]
    df_image.loc[nome_imagem,'Region'] = [x,y,w,h]
    df_image.to_csv(os.path.join(os.getcwd(),r"CSVs\data_images.csv"))
    return [x,y,w,h]

In [ ]:
PVP_start = Image.open(df_image.loc['PVP_start','Path'])
util_gui.locate_image(PVP_start,'PVP_start')

In [15]:
df_image['Region'].astype(object)

Enhance_confirme_enhanceable           NaN
Enhance_confirm_enhanced               NaN
Enhance_disassemble                    NaN
Enhance_enhance                        NaN
Enhance_enhanceable                    NaN
Enhance_enhance_page                   NaN
Enhance_fill                           NaN
Enhance_sort                           NaN
Enhance_tap_to_continue_disassemble    NaN
OpS_battle                             NaN
OpS_collect_ops                        NaN
OpS_friend                             NaN
OpS_guild                              NaN
OpS_helena_meta                        NaN
OpS_info_analysis_renown_meta          NaN
OpS_renown_meta                        NaN
OpS_request_support                    NaN
OpS_start_battle                       NaN
OpS_world                              NaN
PVP_confirm                            NaN
PVP_continue_after_battle              NaN
PVP_continue_merit                     NaN
PVP_exercises                          NaN
PVP_exercis